In [ ]:
"""
Purpose: To put all of the rules together
to autoatically proofread a cell


"""

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from os import sys
sys.path.append("/meshAfterParty/")

In [3]:
from importlib import reload

import neuron_utils as nru

import neuron

import neuron_visualizations as nviz

import time

import datajoint_utils as du

import numpy as np

import proofreading_utils as pru

import preprocessing_vp2 as pre

# -- For the axon classification --

import neuron_searching as ns

import skeleton_utils as sk

import numpy_utils as nu

import networkx_utils as xu

import system_utils as su

import classification_utils as clu
import proofreading_utils as pru

INFO - 2021-02-03 07:54:52,375 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-02-03 07:54:52,376 - settings - Setting database.user to celiib
INFO - 2021-02-03 07:54:52,377 - settings - Setting database.password to newceliipass
INFO - 2021-02-03 07:54:52,388 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-02-03 07:54:52,389 - settings - Setting enable_python_native_blobs to True
INFO - 2021-02-03 07:54:52,401 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-02-03 07:54:52,673 - settings - Setting enable_python_native_blobs to True


In [8]:
minnie,schema = du.configure_minnie_vm()

INFO - 2021-02-03 07:55:03,435 - settings - Setting enable_python_native_blobs to True
INFO - 2021-02-03 07:55:03,733 - settings - Setting enable_python_native_blobs to True


# Importing the Neuron

In [9]:
import datajoint as dj
dj.config["display.limit"] = 40
curr_table = minnie.Decomposition & "process_version = 4" & "n_limbs>5" & "n_error_limbs>0" & "n_somas>1"
curr_table

INFO - 2021-02-03 07:55:03,963 - settings - Setting display.limit to 40


segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),ver the version number of the materializaiton,process_version the version of the preprocessing pipeline run,index the index of the neuron object that resulted from that mesh (indexed starting at 0),multiplicity the number of somas found for this base segment,decomposition,n_vertices number of vertices,n_faces number of faces,n_not_processed_soma_containing_meshes the number of meshes with somas that were not processed,n_error_limbs the number of limbs that are touching multiple somas or 1 soma in multiple places,n_same_soma_multi_touching_limbs number of limbs that touch the same soma multiple times,n_multi_soma_touching_limbs number of limbs that touch multiple somas,n_somas number of soma meshes detected,n_limbs,n_branches,max_limb_n_branches,skeletal_length,max_limb_skeletal_length,median_branch_length gives information on average skeletal length to next branch point,width_median median width from mesh center without spines removed,width_no_spine_median median width from mesh center with spines removed,width_90_perc 90th percentile for width without spines removed,width_no_spine_90_perc 90th percentile for width with spines removed,n_spines,spine_density n_spines/ skeletal_length,spines_per_branch,skeletal_length_eligible the skeletal length for all branches searched for spines,n_spine_eligible_branches the number of branches that were checked for spines because passed width threshold,spine_density_eligible n_spines/skeletal_length_eligible,spines_per_branch_eligible n_spines/n_spine_eligible_branches,total_spine_volume the sum of all spine volume,spine_volume_median median of the spine volume for those spines with able to calculate volume,spine_volume_density total_spine_volume/skeletal_length,spine_volume_density_eligible total_spine_volume/skeletal_length_eligible,spine_volume_per_branch_eligible total_spine_volume/n_spine_eligible_branches,run_time the amount of time to run (seconds)
864691135012398582,0,0.25,30.00,4,0,1,=BLOB=,1376990,2794781,0,3,1,3,2,7,332,193,13857599.090995071,7698499.767603829,24782.936,182.056,175.907,512.504,498.037,5073,0.00036608073062934334,15.28012048192771,9021770.899,184,0.0005623064536655776,27.571,1130755898564.6187,149139625.2549947,81598.25458505328,125336.35704382107,6145412492.199015,18888.6673
864691135065018692,0,0.25,30.00,4,0,1,=BLOB=,927599,1896021,0,2,2,1,2,19,275,62,10134692.441118384,2065744.7931319945,17371.879,110.248,110.248,544.807,544.758,1859,0.0001834293453699376,6.76,5876271.812,132,0.00031635704737206256,14.083,333744826425.14685,116071713.89832559,32930.92793532444,56795.33505301828,2528369897.1602035,13467.9901
864691135065032772,0,0.25,30.00,4,0,1,=BLOB=,1284918,2610256,0,1,0,1,3,19,339,49,12070580.98087547,2067456.6676802342,24476.944,293.07,258.608,496.883,455.767,4680,0.00038771953126489554,13.805309734513274,9141427.307,231,0.0005119550637805012,20.26,984274848135.3904,134477821.95931625,81543.28691343585,107671.90013989249,4260930078.5081835,10872.3472
864691135210078912,0,0.25,30.00,4,0,1,=BLOB=,2254579,4581171,0,1,1,1,3,19,680,410,25709086.238637865,14910186.433055326,22806.349,91.868,91.868,488.977,396.491,7956,0.00030946257389899087,11.7,14370940.599,305,0.0005536172072518077,26.085,1530423053724.611,123769896.05576578,59528.48885871943,106494.2856858726,5017780504.015119,21160.2523
864691135345274271,0,0.25,30.00,4,0,1,=BLOB=,2309163,4637079,0,3,3,1,3,23,568,232,20083889.41475576,7902329.442060159,19837.799,307.913,275.329,552.594,537.217,6717,0.00033444717112737023,11.825704225352112,16093790.786,398,0.00041736593257090944,16.877,1327097700929.9097,126146626.59829712,66077.7239668967,82460.2306924701,3334416333.994748,10629.5786
864691135373436616,0,0.25,30.00,4,0,1,=BLOB=,1785657,3619863,0,5,5,1,2,20,493,131,18421878.018574927,4845043.772362778,26060.468,261.01

In [10]:
segment_id = 864691135654121154 #was very big and hard to visualize
segment_id = 864691135065018692 #worked
segment_id = 864691135526113627 #worked
segment_id = 864691135403709422 #worked
segment_id = 864691136390485119

In [11]:
neuron_obj_original = (curr_table & dict(segment_id=segment_id)).fetch1("decomposition")

Decompressing Neuron in minimal output mode...please wait


In [ ]:
#nviz.visualize_neuron_lite(neuron_obj_original)

In [ ]:
ret_col = nviz.visualize_neuron(neuron_obj_original,
                     limb_branch_dict=dict(L1="all"),
                     return_color_dict=True)

In [ ]:
nviz.plot_soma_limb_concept_network(neuron_obj_original)

# Developing the Overall Proofreading Function

In [43]:
pru = reload(pru)

In [44]:
output_list = pru.proofread_neuron(
    
    input_neuron = neuron_obj_original,

    attempt_to_split_neuron = True,
    plot_neuron_split_results = False,

    plot_neuron_before_filtering = False,

    plot_axon = False,
    plot_axon_like = False,

    # -- for the filtering loop
    plot_limb_branch_filter_with_disconnect_effect = True,
    plot_final_filtered_neuron = True,

    # -- for the output --
    return_process_info = True,

    debug_time = True,
    verbose = False,
    verbose_outline=True

    )

---- Part A: Attempting to split neuron --------



Time for Split Neuron = 159.1078279018402
 # of neurons found after split= 2

--- Working on Neuron 0 ---


 ------ Part B: Axon Classification ---- 


Axon Classification = 23.536929845809937


 ------ Part C: Inhibitory Excitatory Classification ---- 


Did not pass final filters to continuing
Inhibitory Excitatory Classification = 12.031546115875244
cell type = excitatory


 ------ Part D: Neuron Filtering ---- 

 

--- Working on filter 0:
function = axon_on_dendrite_merges
function __name__ = filter_away_axon_on_dendrite_merges
function arguments = {'use_pre_existing_axon_labels': True}




Branches Requested to Remove (WITH disconnect effect)

 Working on visualization type: mesh



--- Working on filter 1:
function = low_branch_clusters
function __name__ = filter_away_low_branch_length_clusters
function arguments = {}



--- Working on filter 2:
function = dendrite_on_axon_merges
function __name__ = filter_away_dendrite_on_axon_merges
function arguments = {'use_pre_existing_axon_labels': True}



--- Working on filter 3:
function = double_back_and_width_change
function __name__ = filter_away_large_double_back_or_width_changes
function arguments = {'perform_double_back_errors': True, 'skip_double_back_errors_for_axon': True, 'double_back_threshold': 140}




Branches Requested to Remove (WITH disconnect effect)

 Working on visualization type: mesh



--- Working on filter 4:
function = high_degree_coordinates
function __name__ = filter_away_high_degree_coordinates
function arguments = {'axon_dependent': True}



--- Working on filter 5:
function = crossovers
function __name__ = filter_away_crossovers
function arguments = {'axon_dependent': True, 'match_threshold': 30}




Branches Requested to Remove (WITH disconnect effect)

 Working on visualization type: mesh





 ---- Total time for applying filter: 100.49672389030457 -----

 Working on visualization type: mesh



--- Working on Neuron 1 ---


 ------ Part B: Axon Classification ---- 


Axon Classification = 27.729567527770996


 ------ Part C: Inhibitory Excitatory Classification ---- 


Inhibitory Excitatory Classification = 18.28194570541382
cell type = inhibitory


 ------ Part D: Neuron Filtering ---- 

 

--- Working on filter 0:
function = low_branch_clusters
function __name__ = filter_away_low_branch_length_clusters
function arguments = {}



--- Working on filter 1:
function = high_degree_coordinates
function __name__ = filter_away_high_degree_coordinates
function arguments = {'axon_dependent': True}



--- Working on filter 2:
function = crossovers
function __name__ = filter_away_crossovers
function arguments = {'axon_dependent': True, 'match_threshold': 30}




Branches Requested to Remove (WITH disconnect effect)

 Working on visualization type: mesh



--- Working on filter 3:
function = double_back_and_width_change
function __name__ = filter_away_large_double_back_or_width_changes
function arguments = {'perform_double_back_errors': False, 'skip_double_back_errors_for_axon': True}





 ---- Total time for applying filter: 74.92204093933105 -----

 Working on visualization type: mesh


In [55]:
neuron_t = neuron.Neuron(output_list[0]["filtered_neuron"])

Recieved another instance of Neuron class in init -- so just copying data


array([17821.07374371,   407.96083255])

In [62]:
neuron_t.get_soma_node_names()

[autoreload of neuron_utils failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/usr/local/lib/python3.6/dist-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/usr/lib/python3.6/imp.py", line 315, in reload
    return importlib.reload(module)
  File "/usr/lib/python3.6/importlib/__init__.py", line 166, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 618, in _exec
  File "<frozen importlib._bootstrap_external>", line 674, in exec_module
  File "<frozen importlib._bootstrap_external>", line 781, in get_code
  File "<frozen importlib._bootstrap_external>", line 741, in source_to_code
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/meshAfterParty/neuron_utils.py", line 4972
    def soma_centers
                 

['S0']

# Figuring Out the Closest Nuclei

In [71]:
m65mat = dj.create_virtual_module('m65mat', 'microns_minnie65_materialization')
nucleus_id,nucleus_x,nucleus_y,nucleus_z = (m65mat.Nucleus.Info & {'ver': 30} & dict(segment_id=neuron_t.segment_id)).fetch("nucleus_id",
                                                                               "nucleus_x",
                                                                               "nucleus_y",
                                                                              "nucleus_z")
nucleus_centers = np.vstack([nucleus_x,nucleus_y,nucleus_z]).T

np.linalg.norm(nucleus_centers - nru.soma_centers(neuron_t,),axis=1)

array([[237104, 195056,  17137],
       [254592, 194656,  16781]])

In [61]:
minnie.DecompositionSplit() & "split_success>0"

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),ver the version number of the materializaiton,process_version the version of the preprocessing pipeline run,index the index of the neuron object that resulted from that mesh (indexed starting at 0),split_index the index of the neuron object that resulted AFTER THE SPLITTING ALGORITHM,split_version the version of the splitting algorithm used,multiplicity the number of somas found for this base segment,n_splits the number of cuts required to help split the neuron,split_success the successfulness of the splitting,n_error_limbs_cancelled number of limbs that couldn't be resolved and cancelled out,n_same_soma_limbs_cancelled number of same soma touching limbs that couldn't be resolved and cancelled out,n_multi_soma_limbs_cancelled number of multi soma touching limbs that couldn't be resolved and cancelled out,error_imbs_cancelled_area the total area (in microns^2) of the limbs that was cancelled out because touching the same soma multiple times or multiple somas,error_imbs_cancelled_skeletal_length the total skeletal length (in microns) of the limbs that were called out because could not be resolved,split_results will store the results of how to split the limbs of neuron objects from original neuron,decomposition,n_vertices number of vertices,n_faces number of faces,n_not_processed_soma_containing_meshes the number of meshes with somas that were not processed,n_error_limbs the number of limbs that are touching multiple somas or 1 soma in multiple places,n_same_soma_multi_touching_limbs number of limbs that touch the same soma multiple times,n_multi_soma_touching_limbs number of limbs that touch multiple somas,n_somas number of soma meshes detected,max_soma_n_faces The largest number of faces of the somas,max_soma_volume The largest volume of the somas the (volume in billions (10*9 nm^3)),n_limbs,n_branches,max_limb_n_branches,skeletal_length,max_limb_skeletal_length,median_branch_length gives information on average skeletal length to next branch point,width_median median width from mesh center without spines removed,width_no_spine_median median width from mesh center with spines removed,width_90_perc 90th percentile for width without spines removed,width_no_spine_90_perc 90th percentile for width with spines removed,n_spines,spine_density n_spines/ skeletal_length,spines_per_branch,skeletal_length_eligible the skeletal length for all branches searched for spines,n_spine_eligible_branches the number of branches that were checked for spines because passed width threshold,spine_density_eligible n_spines/skeletal_length_eligible,spines_per_branch_eligible n_spines/n_spine_eligible_branches,total_spine_volume the sum of all spine volume,spine_volume_median median of the spine volume for those spines with able to calculate volume,spine_volume_density total_spine_volume/skeletal_length,spine_volume_density_eligible total_spine_volume/skeletal_length_eligible,spine_volume_per_branch_eligible total_spine_volume/n_spine_eligible_branches,run_time the amount of time to run (seconds)


In [76]:
nucleus_id

array([400459, 431655])

In [75]:
#has the 
m65mat.NucleusSynapse()

ver materialization version,synapse_id synapse index within the segmentation,presyn segment_id of the presynaptic cell. Equivalent to Allen 'pt_root_id'.,postsyn segment_id of the postsynaptic cell. Equivalent to Allen 'pt_root_id'.,"synapse_x x coordinate of synapse centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm). From Allen 'ctr_pt_position'.","synapse_y y coordinate of centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm). From Allen 'ctr_pt_position'.","synapse_z z coordinate of centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm). From Allen 'ctr_pt_position'.",synapse_size (EM voxels) scaled by (4x4x40)
30.00,10957,864691135087399350,864691135087399350,55164,103758,21202,20484
30.00,11021,864691135087399350,864691135087399350,55242,102912,21202,4780
30.00,11047,864691135087399350,864691135087399350,55336,102653,21202,3368
30.00,11065,864691135087399350,864691135087399350,55397,103990,21199,628
30.00,11075,864691135087399350,864691135087399350,55429,103423,21201,6292
30.00,11081,864691135087399350,864691135087399350,55444,103664,21203,2932
30.00,11203,864691135087399350,864691135087399350,55786,104910,21208,14656
30.00,11220,864691135087399350,864691135087399350,55778,102596,21203,2840
30.00,11240,864691135087399350,864691135087399350,56040,104665,21199,16628
30.00,11338,864691135087399350,864691135087399350,56036,101908,21200,264


In [78]:
minnie.Decomposition() & "n_somas > 3"

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),ver the version number of the materializaiton,process_version the version of the preprocessing pipeline run,index the index of the neuron object that resulted from that mesh (indexed starting at 0),multiplicity the number of somas found for this base segment,decomposition,n_vertices number of vertices,n_faces number of faces,n_not_processed_soma_containing_meshes the number of meshes with somas that were not processed,n_error_limbs the number of limbs that are touching multiple somas or 1 soma in multiple places,n_same_soma_multi_touching_limbs number of limbs that touch the same soma multiple times,n_multi_soma_touching_limbs number of limbs that touch multiple somas,n_somas number of soma meshes detected,n_limbs,n_branches,max_limb_n_branches,skeletal_length,max_limb_skeletal_length,median_branch_length gives information on average skeletal length to next branch point,width_median median width from mesh center without spines removed,width_no_spine_median median width from mesh center with spines removed,width_90_perc 90th percentile for width without spines removed,width_no_spine_90_perc 90th percentile for width with spines removed,n_spines,spine_density n_spines/ skeletal_length,spines_per_branch,skeletal_length_eligible the skeletal length for all branches searched for spines,n_spine_eligible_branches the number of branches that were checked for spines because passed width threshold,spine_density_eligible n_spines/skeletal_length_eligible,spines_per_branch_eligible n_spines/n_spine_eligible_branches,total_spine_volume the sum of all spine volume,spine_volume_median median of the spine volume for those spines with able to calculate volume,spine_volume_density total_spine_volume/skeletal_length,spine_volume_density_eligible total_spine_volume/skeletal_length_eligible,spine_volume_per_branch_eligible total_spine_volume/n_spine_eligible_branches,run_time the amount of time to run (seconds)
864691134884798458,0,0.25,30.00,5,0,1,=BLOB=,1248054,2518219,0,2,2,1,4,21,307,113,9897647.588574875,4080795.6366048004,20497.234,261.811,249.587,495.206,451.155,4315,0.00043596217802106054,14.055374592833877,8326432.661,221,0.0005182291355349496,19.525,1079382551883.7961,159347648.98170978,109054.45382090153,129633.25301836559,4884083945.175549,4983.3048
864691134884798714,0,0.25,30.00,5,0,1,=BLOB=,2671415,5357446,0,2,1,2,4,27,544,235,23321542.28173015,9038558.768863926,27990.086,334.115,290.072,563.091,536.242,9744,0.00041781113282689486,17.91176470588235,18795445.239,386,0.0005184234731391989,25.244,1873647718976.5208,115368240.39381662,80339.78612316377,99686.2641534427,4854009634.654199,13839.0081
864691134917433610,0,0.25,30.00,5,0,1,=BLOB=,1795986,3629115,0,2,2,2,4,25,428,89,15199035.710229188,3301325.376823318,24286.778,334.025,286.834,562.047,544.444,7107,0.00046759545378374746,16.60514018691589,12416513.031,326,0.000572382921215975,21.801,1302156330907.5764,109003988.9123179,85673.61480907667,104872.9484402356,3994344573.336124,8043.6529
864691134940016227,0,0.25,30.00,5,0,1,=BLOB=,2404940,4905376,0,4,3,4,5,26,620,166,22460012.175845955,4950913.476524887,19835.721,203.398,204.512,526.079,494.231,8057,0.00035872643064123954,12.995161290322581,15247712.112,397,0.0005284071433680279,20.295,1683685181601.816,126968834.53933588,74963.68071485251,110422.15181100843,4241020608.5688057,12603.0705
864691134988486778,0,0.25,30.00,5,0,1,=BLOB=,1517956,3102675,0,2,0,2,4,26,287,73,14216756.266969433,3031021.50984452,31212.205,293.687,261.013,512.45,473.882,5658,0.00039798107906974115,19.714285714285715,12080150.93,217,0.00046837163151238875,26.074,1216721113790.6777,148745191.0488122,85583.59522682065,100720.68808089617,5607009740.970865,6730.0307
864691134988488570,0,0.25,30.00,5,0,1,=BLOB=,1751752,3591099,0,2,1,2,4,21,501,123,15478205.097229742,301083

In [110]:
len(minnie.SpineRecalculation())

11770

In [89]:
segment_id=864691135113193881

In [90]:
du.need_spine_recalculation(segment_id)

False

In [108]:
du.recalculate_spines_with_neuron(neuron_obj,split_index = 0,
                                 verbose=True)

Key for spine recalculation = {'segment_id': 864691135113193881, 'decimation_version': 0, 'decimation_ratio': Decimal('0.25'), 'ver': Decimal('30.00'), 'process_version': 5, 'index': 0, 'split_index': 0, 'split_version': 1}


In [103]:
neuron_obj = (minnie.DecompositionSplit() & dict(segment_id=864691135113193881) & "split_index=0").fetch1("decomposition")
neuron_obj

Decompressing Neuron in minimal output mode...please wait


In [105]:
nviz.visualize_neuron(neuron_obj,
                     limb_branch_dict="all")


 Working on visualization type: mesh

 Working on visualization type: skeleton


In [143]:
curr_table = (minnie.Decomposition & "process_version < 3" & "n_limbs > 4" & "n_spines > 20") - minnie.SpineRecalculation.proj()
minnie.DecompositionSplit() & curr_table.proj()

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),ver the version number of the materializaiton,process_version the version of the preprocessing pipeline run,index the index of the neuron object that resulted from that mesh (indexed starting at 0),split_index the index of the neuron object that resulted AFTER THE SPLITTING ALGORITHM,split_version the version of the splitting algorithm used,multiplicity the number of somas found for this base segment,n_splits the number of cuts required to help split the neuron,split_success the successfulness of the splitting,n_error_limbs_cancelled number of limbs that couldn't be resolved and cancelled out,n_same_soma_limbs_cancelled number of same soma touching limbs that couldn't be resolved and cancelled out,n_multi_soma_limbs_cancelled number of multi soma touching limbs that couldn't be resolved and cancelled out,error_imbs_cancelled_area the total area (in microns^2) of the limbs that was cancelled out because touching the same soma multiple times or multiple somas,error_imbs_cancelled_skeletal_length the total skeletal length (in microns) of the limbs that were called out because could not be resolved,split_results will store the results of how to split the limbs of neuron objects from original neuron,decomposition,n_vertices number of vertices,n_faces number of faces,n_not_processed_soma_containing_meshes the number of meshes with somas that were not processed,n_error_limbs the number of limbs that are touching multiple somas or 1 soma in multiple places,n_same_soma_multi_touching_limbs number of limbs that touch the same soma multiple times,n_multi_soma_touching_limbs number of limbs that touch multiple somas,n_somas number of soma meshes detected,max_soma_n_faces The largest number of faces of the somas,max_soma_volume The largest volume of the somas the (volume in billions (10*9 nm^3)),n_limbs,n_branches,max_limb_n_branches,skeletal_length,max_limb_skeletal_length,median_branch_length gives information on average skeletal length to next branch point,width_median median width from mesh center without spines removed,width_no_spine_median median width from mesh center with spines removed,width_90_perc 90th percentile for width without spines removed,width_no_spine_90_perc 90th percentile for width with spines removed,n_spines,spine_density n_spines/ skeletal_length,spines_per_branch,skeletal_length_eligible the skeletal length for all branches searched for spines,n_spine_eligible_branches the number of branches that were checked for spines because passed width threshold,spine_density_eligible n_spines/skeletal_length_eligible,spines_per_branch_eligible n_spines/n_spine_eligible_branches,total_spine_volume the sum of all spine volume,spine_volume_median median of the spine volume for those spines with able to calculate volume,spine_volume_density total_spine_volume/skeletal_length,spine_volume_density_eligible total_spine_volume/skeletal_length_eligible,spine_volume_per_branch_eligible total_spine_volume/n_spine_eligible_branches,run_time the amount of time to run (seconds)
864691134868397058,0,0.25,30.00,0,0,0,1,1,0,0,0,0,0,0.0,0.0,=BLOB=,=BLOB=,125239,252340,0,0,0,0,1,20060,163,5,63,36,837263.6265405079,497656.8367177575,12695.808,133.911,133.911,207.467,207.467,107,0.00012779726314173426,1.6984126984126984,434341.17,29,0.0002463501214955055,3.69,19453277521.483112,60676115.38333638,23234.351648431413,44788.01197105748,670802673.1545901,16.9201
864691134884742394,0,0.25,30.00,0,0,0,1,1,1,0,0,0,0,0.0,0.0,=BLOB=,=BLOB=,180848,360013,0,0,0,0,1,25557,470,8,71,43,3555740.0658564987,1553147.4052075923,35053.668,93.767,93.767,277.293,274.39,167,4.6966312752609323e-05,2.352112676056338,1452675.168,23,0.00011496031850666287,7.261,40705481816.736855,146565582.96327972,11447.822693116857,28021.048830055344,1769803557.2494285,236.1091
86469113488474265

In [144]:
segment_id = 864691134917418250

In [145]:
minnie.SpineRecalculation & dict(segment_id=segment_id)

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),ver the version number of the materializaiton,process_version the version of the preprocessing pipeline run,index the index of the neuron object that resulted from that mesh (indexed starting at 0),split_index the index of the neuron object that resulted AFTER THE SPLITTING ALGORITHM,spine_version the version of the spine algorithm,"updated_spines whether or not the spines were updated (1 = yes, 0 = no)",n_spines_old number of spines before recalculation,n_spines_new number of spines after recalculation,spine_data stores the newly computes spines that were used for the classification,run_time the amount of time to run (seconds)


In [146]:
neuron_to_recal = (curr_table & dict(segment_id=segment_id))#.fetch1("decomposition")
neuron_to_recal

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),ver the version number of the materializaiton,process_version the version of the preprocessing pipeline run,index the index of the neuron object that resulted from that mesh (indexed starting at 0),multiplicity the number of somas found for this base segment,decomposition,n_vertices number of vertices,n_faces number of faces,n_not_processed_soma_containing_meshes the number of meshes with somas that were not processed,n_error_limbs the number of limbs that are touching multiple somas or 1 soma in multiple places,n_same_soma_multi_touching_limbs number of limbs that touch the same soma multiple times,n_multi_soma_touching_limbs number of limbs that touch multiple somas,n_somas number of soma meshes detected,n_limbs,n_branches,max_limb_n_branches,skeletal_length,max_limb_skeletal_length,median_branch_length gives information on average skeletal length to next branch point,width_median median width from mesh center without spines removed,width_no_spine_median median width from mesh center with spines removed,width_90_perc 90th percentile for width without spines removed,width_no_spine_90_perc 90th percentile for width with spines removed,n_spines,spine_density n_spines/ skeletal_length,spines_per_branch,skeletal_length_eligible the skeletal length for all branches searched for spines,n_spine_eligible_branches the number of branches that were checked for spines because passed width threshold,spine_density_eligible n_spines/skeletal_length_eligible,spines_per_branch_eligible n_spines/n_spine_eligible_branches,total_spine_volume the sum of all spine volume,spine_volume_median median of the spine volume for those spines with able to calculate volume,spine_volume_density total_spine_volume/skeletal_length,spine_volume_density_eligible total_spine_volume/skeletal_length_eligible,spine_volume_per_branch_eligible total_spine_volume/n_spine_eligible_branches,run_time the amount of time to run (seconds)
864691134917418250,0,0.25,30.00,0,0,1,=BLOB=,1087424,2201968,0,2,1,1,2,15,290,126,10972952.971804027,4049148.8828560673,25651.201,110.62,110.62,487.671,431.492,4672,0.0004257741751017358,16.110344827586207,6545878.694,137,0.000713731527637747,34.102,853298415280.7451,108280014.25405629,77763.79042846269,130356.58849939957,6228455585.99084,5337.9898


In [147]:
minnie.DecompositionSplit() & dict(segment_id=segment_id)

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),ver the version number of the materializaiton,process_version the version of the preprocessing pipeline run,index the index of the neuron object that resulted from that mesh (indexed starting at 0),split_index the index of the neuron object that resulted AFTER THE SPLITTING ALGORITHM,split_version the version of the splitting algorithm used,multiplicity the number of somas found for this base segment,n_splits the number of cuts required to help split the neuron,split_success the successfulness of the splitting,n_error_limbs_cancelled number of limbs that couldn't be resolved and cancelled out,n_same_soma_limbs_cancelled number of same soma touching limbs that couldn't be resolved and cancelled out,n_multi_soma_limbs_cancelled number of multi soma touching limbs that couldn't be resolved and cancelled out,error_imbs_cancelled_area the total area (in microns^2) of the limbs that was cancelled out because touching the same soma multiple times or multiple somas,error_imbs_cancelled_skeletal_length the total skeletal length (in microns) of the limbs that were called out because could not be resolved,split_results will store the results of how to split the limbs of neuron objects from original neuron,decomposition,n_vertices number of vertices,n_faces number of faces,n_not_processed_soma_containing_meshes the number of meshes with somas that were not processed,n_error_limbs the number of limbs that are touching multiple somas or 1 soma in multiple places,n_same_soma_multi_touching_limbs number of limbs that touch the same soma multiple times,n_multi_soma_touching_limbs number of limbs that touch multiple somas,n_somas number of soma meshes detected,max_soma_n_faces The largest number of faces of the somas,max_soma_volume The largest volume of the somas the (volume in billions (10*9 nm^3)),n_limbs,n_branches,max_limb_n_branches,skeletal_length,max_limb_skeletal_length,median_branch_length gives information on average skeletal length to next branch point,width_median median width from mesh center without spines removed,width_no_spine_median median width from mesh center with spines removed,width_90_perc 90th percentile for width without spines removed,width_no_spine_90_perc 90th percentile for width with spines removed,n_spines,spine_density n_spines/ skeletal_length,spines_per_branch,skeletal_length_eligible the skeletal length for all branches searched for spines,n_spine_eligible_branches the number of branches that were checked for spines because passed width threshold,spine_density_eligible n_spines/skeletal_length_eligible,spines_per_branch_eligible n_spines/n_spine_eligible_branches,total_spine_volume the sum of all spine volume,spine_volume_median median of the spine volume for those spines with able to calculate volume,spine_volume_density total_spine_volume/skeletal_length,spine_volume_density_eligible total_spine_volume/skeletal_length_eligible,spine_volume_per_branch_eligible total_spine_volume/n_spine_eligible_branches,run_time the amount of time to run (seconds)
864691134917418250,0,0.25,30.00,0,0,0,1,2,2,0,0,0,0,0.0,0.0,=BLOB=,=BLOB=,664869,1326425,0,0,0,0,1,37101,727,9,235,126,8176410.148964731,4049148.8828560673,24064.622,83.178,83.178,465.201,373.006,2834,0.0003466068786139393,12.059574468085106,3832148.441,87,0.0007395329391938865,32.575,426661885843.27374,91120606.1001571,52182.055213717016,111337.51534216045,4904159607.393951,2870.4111
864691134917418250,0,0.25,30.00,0,0,1,1,2,2,0,0,0,0,0.0,0.0,=BLOB=,=BLOB=,377713,753627,0,0,0,0,1,35176,681,8,54,18,2795845.117069319,913183.6880837162,39779.348,400.571,305.867,530.253,520.556,1838,0.0006574040846463776,34.03703703703704,2714591.048,50,0.0006770817288866268,36.76,426636529437.4713,142082310.3701706,152596.62519670738,157164.1996505513,8532730588.749426,3962.9953


In [151]:
neuron_objs,neuron_split_idxs = du.decomposition_with_spine_recalculation(segment_id)

Decompressing Neuron in minimal output mode...please wait
Decompressing Neuron in minimal output mode...please wait
Number of Neurons found = 2


In [152]:
neuron_split_idxs

[0, 1]

In [156]:
neuron_obj = neuron_objs[0]
neuron_obj.n_spines

2590

In [157]:
nviz.visualize_neuron(neuron_obj,
                      limb_branch_dict="all")


 Working on visualization type: mesh

 Working on visualization type: skeleton


In [158]:
import proofreading_utils as pru
pru.proofread_neuron(neuron_obj,
                    plot_limb_branch_filter_with_disconnect_effect=True,
                    plot_final_filtered_neuron=True)

---- Part A: NOT Attempting to split neuron --------

--- Working on Neuron 0 ---


 ------ Part B: Axon Classification ---- 


Axon Classification = 36.21621251106262


 ------ Part C: Inhibitory Excitatory Classification ---- 


spine_category = densely_spined
Inhibitory Excitatory Classification = 14.948839664459229

 -- Cell Type Classification Results --
inh_exc_class=excitatory
spine_category=densely_spined
axon_angles={1: {0: 169.1538366300066}}
n_axons=1
n_apicals=1
neuron_spine_density=0.0008681729613583276
n_branches_processed=46
skeletal_length_processed=2581869.9693673705
n_branches_in_search_radius=77
skeletal_length_in_search_radius=3117611.3628649455


 ------ Part D: Neuron Filtering ---- 

 

--- Working on filter 0:
function = axon_on_dendrite_merges
function __name__ = filter_away_axon_on_dendrite_merges
function arguments = {'use_pre_existing_axon_labels': True}


----------------------- Running Filter ------------------
using pre-existing labels for axon-error dete

total_sk_distance = 916.5990925955376, total_area = 772.9502686946934
----------------------- FINISHED Running Filter ------------------



 --Filter axon_on_dendrite_merges Results --
local_results = {'axon_on_dendrite_merges_neuron': <neuron.Neuron object at 0x7f7299c434e0>, 'axon_on_dendrite_merges_time': 33.359355211257935, 'axon_on_dendrite_merges_error_area': 772.9502686946934, 'axon_on_dendrite_merges_error_length': 916.5990925955376}

--- Working on filter 1:
function = low_branch_clusters
function __name__ = filter_away_low_branch_length_clusters
function arguments = {}


----------------------- Running Filter ------------------
limb_branch_restriction = {'L0': array([7]), 'L1': array([ 18,  29,  34,  45,  53,  62,  64, 100])}
--- Working on Limb L0 ---
nodes_to_keep = [7]
--- Working on Limb L1 ---
nodes_to_keep = [ 18  29  34  45  53  62  64 100]
limb_branch_dict was empty so returning original neuron
----------------------- FINISHED Running Filter ------------------



 --F

total_sk_distance = 12.610170962932338, total_area = 21.751433402308823
----------------------- FINISHED Running Filter ------------------



 --Filter double_back_and_width_change Results --
local_results = {'double_back_and_width_change_neuron': <neuron.Neuron object at 0x7f729c5a3320>, 'double_back_and_width_change_time': 89.24971961975098, 'double_back_and_width_change_error_area': 21.751433402308823, 'double_back_and_width_change_error_length': 12.610170962932338}

--- Working on filter 4:
function = crossovers
function __name__ = filter_away_crossovers
function arguments = {'axon_dependent': True, 'match_threshold': 30}


----------------------- Running Filter ------------------
crossover_coordinates = []

--- Working on Limb L0 ---
edges_to_create = []
edges_to_create = []
crossover_coordinates = [[517413.6491809  797064.64663192 969359.25999295]
 [529836.78605875 790186.76718022 969802.52035515]
 [532781.11029465 825917.50433483 935133.49146412]
 [554857.         811543.       

total_sk_distance = 310.23749334689063, total_area = 252.9867014966439
----------------------- FINISHED Running Filter ------------------



 --Filter crossovers Results --
local_results = {'crossovers_neuron': <neuron.Neuron object at 0x7f729bf95898>, 'crossovers_time': 39.641578912734985, 'crossovers_error_area': 252.9867014966439, 'crossovers_error_length': 310.23749334689063}

--- Working on filter 5:
function = high_degree_coordinates
function __name__ = filter_away_high_degree_coordinates
function arguments = {'axon_dependent': True, 'min_degree_to_find': 4}


----------------------- Running Filter ------------------
curr_high_degree_nodes for get_nodes_greater_or_equal_degree_k = []
high_degree_branch_groups = []

--- Working on Limb L0 ---
edges_to_create = []
edges_to_create = []
curr_high_degree_nodes for get_nodes_greater_or_equal_degree_k = [9541, 9828, 24414, 29802]
curr_high_degree_coordinates = [[517413.6491809  797064.64663192 969359.25999295]
 [518088.21710352 782145.9

total_sk_distance = 1995.145339391494, total_area = 2061.2894470814053
----------------------- FINISHED Running Filter ------------------



 --Filter high_degree_coordinates Results --
local_results = {'high_degree_coordinates_neuron': <neuron.Neuron object at 0x7f72995f74e0>, 'high_degree_coordinates_time': 26.12311887741089, 'high_degree_coordinates_error_area': 2061.2894470814053, 'high_degree_coordinates_error_length': 1995.145339391494}



 ---- Total time for applying filter: 190.25700211524963 -----

               ******* Plotting Final Neuron ***************

 Working on visualization type: mesh




 ------ Part E: Save Neuron  ---- 

 


{'filtered_neuron': <neuron.Neuron at 0x7f72995f74e0>,
 'split_info': {'errored_limbs_area': [],
  'errored_limbs_skeletal_length': [],
  'n_multi_soma_errors': 0,
  'n_same_soma_errors': 0},
 'cell_type_info': {'inh_exc_class': 'excitatory',
  'spine_category': 'densely_spined',
  'axon_angles': {1: {0: 169.1538366300066}},
  'axon_angle_maximum': 169.1538366300066,
  'n_axons': 1,
  'n_apicals': 1,
  'neuron_spine_density': 0.0008681729613583276,
  'n_branches_processed': 0.0008681729613583276,
  'skeletal_length_processed': 2581869.9693673705,
  'n_branches_in_search_radius': 77,
  'skeletal_length_in_search_radius': 3117611.3628649455},
 'filtering_info': {'axon_on_dendrite_merges_neuron': <neuron.Neuron at 0x7f7299c434e0>,
  'axon_on_dendrite_merges_time': 33.359355211257935,
  'axon_on_dendrite_merges_error_area': 772.9502686946934,
  'axon_on_dendrite_merges_error_length': 916.5990925955376,
  'low_branch_clusters_neuron': <neuron.Neuron at 0x7f72ae0d3278>,
  'low_branch_cluster

In [ ]:
"""




"""

In [124]:
current_neuron = (minnie.DecompositionSplit & dict(segment_id=segment_id)).fetch1("decomposition")
current_neuron

Decompressing Neuron in minimal output mode...please wait


In [126]:
nviz.visualize_neuron(current_neuron,
                     limb_branch_dict="all")


 Working on visualization type: mesh

 Working on visualization type: skeleton


In [132]:
 minnie.SpineRecalculation()

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),ver the version number of the materializaiton,process_version the version of the preprocessing pipeline run,index the index of the neuron object that resulted from that mesh (indexed starting at 0),split_index the index of the neuron object that resulted AFTER THE SPLITTING ALGORITHM,spine_version the version of the spine algorithm,"updated_spines whether or not the spines were updated (1 = yes, 0 = no)",n_spines_old number of spines before recalculation,n_spines_new number of spines after recalculation,spine_data stores the newly computes spines that were used for the classification,run_time the amount of time to run (seconds)
864691134384033872,0,0.25,30.00,0,0,0,0,1,0,0,=BLOB=,15.8765
864691134428609977,0,0.25,30.00,0,0,0,0,1,0,0,=BLOB=,13.5525
864691134441691943,0,0.25,30.00,0,0,0,0,1,0,0,=BLOB=,14.4608
864691134484023255,0,0.25,30.00,0,0,0,0,1,0,0,=BLOB=,16.0516
864691134488295985,0,0.25,30.00,0,0,0,0,1,0,0,=BLOB=,16.7391
864691134510995985,0,0.25,30.00,0,0,0,0,1,0,0,=BLOB=,15.6807
864691134570884889,0,0.25,30.00,0,0,0,0,1,0,0,=BLOB=,14.7505
864691134577177340,0,0.25,30.00,0,0,0,0,1,0,0,=BLOB=,20.8762
864691134594423629,0,0.25,30.00,0,0,0,0,1,0,0,=BLOB=,14.5686
864691134655557709,0,0.25,30.00,0,0,0,0,1,5,5,=BLOB=,30.5942
